<a href="https://colab.research.google.com/github/nana-lyj/BilingualChildEmo/blob/main/Qwen2.5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. 安装依赖
!pip install -q transformers datasets accelerate peft bitsandbytes unsloth

# 2. 初始化设置
from unsloth import FastLanguageModel
import torch
from transformers import TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset
import pandas as pd

max_seq_length = 1024
model_name = "unsloth/Qwen2.5-7B"
dtype = torch.float16
load_in_4bit = True

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# 3. 加载模型（使用Unsloth优化）
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/106k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [3]:
# 4. 添加LoRA适配器
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
)


Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 0 MLP layers.


In [4]:
# 5. 加载并预处理数据
dataset = load_dataset("nanaaaa/BilingualChildEmo")

README.md:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

BilingualChildEmo.py:   0%|          | 0.00/3.04k [00:00<?, ?B/s]

The repository for nanaaaa/BilingualChildEmo contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/nanaaaa/BilingualChildEmo.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
# 定义分类模板
classification_template = """### Instruction:
分析文本情感并从[joy, sadness, anger, fear, love]中选择最合适的类别。

### Input:
{text}

### Response:
{label}"""



In [6]:
# 格式化函数
def format_data(examples):
    label_map = {0: "joy", 1: "sadness", 2: "anger", 3: "fear", 4: "love"}
    texts = []
    for sentence, label in zip(examples["sentence"], examples["label"]):
        formatted = classification_template.format(
            text=sentence,
            label=label_map[label]
        ) + tokenizer.eos_token
        texts.append(formatted)
    return {"text": texts}

# 应用格式化
dataset = dataset.map(format_data, batched=True)

Map:   0%|          | 0/1524 [00:00<?, ? examples/s]

Map:   0%|          | 0/326 [00:00<?, ? examples/s]

Map:   0%|          | 0/327 [00:00<?, ? examples/s]

In [7]:
# 6. 训练配置
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    eval_dataset = dataset["validation"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        output_dir = "./results",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 3,
        learning_rate = 2e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        evaluation_strategy = "steps",
        eval_steps = 200,
        save_strategy = "steps",
        save_steps = 200,
    ),
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1524 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/326 [00:00<?, ? examples/s]

In [8]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,524 | Num Epochs = 3 | Total steps = 570
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 10,092,544/7,000,000,000 (0.14% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yanjin-liu (yanjin-liu-cityu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
200,1.398600,1.442036
400,1.511500,1.401013


Unsloth: Will smartly offload gradients to save VRAM!


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=570, training_loss=1.6555733178791248, metrics={'train_runtime': 1722.9691, 'train_samples_per_second': 2.654, 'train_steps_per_second': 0.331, 'total_flos': 1.288131565679616e+16, 'train_loss': 1.6555733178791248})

In [9]:
# 8. 保存模型
model.save_pretrained("classifier_model")
tokenizer.save_pretrained("classifier_model")

('classifier_model/tokenizer_config.json',
 'classifier_model/special_tokens_map.json',
 'classifier_model/vocab.json',
 'classifier_model/merges.txt',
 'classifier_model/added_tokens.json',
 'classifier_model/tokenizer.json')

In [21]:
# 9. 推理函数
def predict_emotion(text):
    prompt = classification_template.format(text=text, label="").strip()
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=10,
        temperature=0.01,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
    )

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = generated.split("### Response:")[-1].strip().lower()

    label_map = {"joy": 0, "sadness": 1, "anger": 2, "fear": 3, "love": 4}
    return label_map.get(response, -1)  # 返回-1表示预测失败


In [10]:
from sklearn.metrics import classification_report, f1_score
import json

In [22]:
# 10. 在测试集上评估模型性能
def evaluate_on_test_set():
    test_data = dataset["test"]
    test_sentences = test_data["sentence"]
    true_labels = test_data["label"]

    # 批量预测
    predicted_labels = []
    for sentence in test_sentences:
        predicted_labels.append(predict_emotion(sentence))
        # 过滤掉预测失败的样本
    valid_indices = [i for i, label in enumerate(predicted_labels) if label != -1]
    filtered_true = [true_labels[i] for i in valid_indices]
    filtered_pred = [predicted_labels[i] for i in valid_indices]

    if len(filtered_true) == 0:
        print("所有预测都失败了，无法计算评估指标")
        return

    # 计算评估指标
    print("\n测试集评估结果:")
    print(classification_report(
        filtered_true,
        filtered_pred,
        target_names=["joy", "sadness", "anger", "fear", "love"],
        digits=4
    ))

    # 计算F1分数
    f1_micro = f1_score(filtered_true, filtered_pred, average='micro')
    f1_macro = f1_score(filtered_true, filtered_pred, average='macro')
    f1_weighted = f1_score(filtered_true, filtered_pred, average='weighted')

    print(f"\nF1 Micro: {f1_micro:.4f}")
    print(f"F1 Macro: {f1_macro:.4f}")
    print(f"F1 Weighted: {f1_weighted:.4f}")

    # 保存评估结果
    results = {
        "f1_micro": f1_micro,
        "f1_macro": f1_macro,
        "f1_weighted": f1_weighted,
        "classification_report": classification_report(
            filtered_true,
            filtered_pred,
            target_names=["joy", "sadness", "anger", "fear", "love"],
            output_dict=True
        )
    }

    with open("evaluation_results.json", "w") as f:
        json.dump(results, f, indent=4)

    print("\n评估结果已保存到 evaluation_results.json")

# 执行评估
evaluate_on_test_set()



测试集评估结果:
              precision    recall  f1-score   support

         joy     0.7638    0.9417    0.8435       103
     sadness     0.7333    0.5500    0.6286        80
       anger     0.8235    0.4118    0.5490        34
        fear     0.4940    0.8542    0.6260        48
        love     1.0000    0.4211    0.5926        38

    accuracy                         0.6997       303
   macro avg     0.7629    0.6357    0.6479       303
weighted avg     0.7493    0.6997    0.6878       303


F1 Micro: 0.6997
F1 Macro: 0.6479
F1 Weighted: 0.6878

评估结果已保存到 evaluation_results.json
